## Imports

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pandas as pd
from PIL import Image

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers

## EDA (Exploratory Data Analysis)

In [ ]:
# Set the directory path where the images are located
directory_path = os.environ['DATASET_PATH']

# Get a list of all the image files in the directory
image_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.jpg')]

# Sort the list of image files by name (optional)
image_files.sort()

# Create a subplot grid with 4 rows and 5 columns
fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(10, 8))

# Loop through the first 20 images and display them in the subplot grid
for i in range(20):
    if i >= len(image_files):
        break
    img = imread(image_files[i])
    row = i // 5
    col = i % 5
    axs[row, col].imshow(img)
    axs[row, col].axis('off')

# Display the subplot grid
plt.show()


In [ ]:
print ('Number of images:', len(image_files))

In [ ]:
path = os.environ['CSVPATH']

# Load data from CSV file
df = pd.read_csv(path)

classes = {-1: 'Female', 1: 'Male'}

plt.hist(list(map(lambda x: classes[x] , df['Male'])))
plt.show()

In [ ]:
classes = {-1: 'Old', 1: 'Young'}

plt.hist(list(map(lambda x: classes[x] , df['Young'])))
plt.show()

## Hyperparameters

In [ ]:
EPOCHS = 100
NOISE_DIM = 250
BATCH_SIZE = 64

MAX_SIZE = len(image_files)

# Select percentage of data to use in each epoch
RANDOM_SELECT = 1

DISCRIMINATOR_ACCURACY_THRESHOLD = 0.7

INTERVAL = 5

## Models

### Discriminator

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(128, 128, 3,)),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])

    return model

In [ ]:
discriminator = make_discriminator_model()
discriminator.summary()

### Generator

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(32*32*512, use_bias=False, input_shape=(NOISE_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((32, 32, 512)),
        layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh')
    ])

    return model

In [ ]:
generator = make_generator_model()
generator.summary()

## Loss Functions and Optimizers

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)
loss_object = tf.keras.losses.BinaryCrossentropy()

### Continute Training

In [ ]:
TRAINING_RESUME = False
epoch = 0

if(TRAINING_RESUME):
    generator.load_weights(os.path.join(os.environ['CHECKPOINT_PATH'], f"generator_epoch_{epoch}.h5"))
    discriminator.load_weights(os.path.join(os.environ['CHECKPOINT_PATH'], f"discriminator_epoch_{epoch}.h5"))

In [ ]:
## Discriminator setup
discriminator.compile(optimizer=optimizer, loss=loss_object, metrics=['accuracy'])
discriminator.trainable = False

In [ ]:
## Generator setup
noise = layers.Input(shape=(NOISE_DIM,))
image = generator(noise)

## Combined model
validity = discriminator(image)
combined = tf.keras.Model(noise, validity)
combined.compile(optimizer=optimizer, loss=loss_object)

## Utility Functions

In [ ]:
def discriminator_train(current_accuracy):
    return current_accuracy < DISCRIMINATOR_ACCURACY_THRESHOLD

In [ ]:
def sample_image(epoch):
    noise = np.random.normal(0, 1, (1, NOISE_DIM))

    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig = gen_imgs[0]
    im = Image.fromarray((fig * 255).astype(np.uint8))

    path = os.path.join(os.environ['SAMPLES_PATH'], f"image_{epoch}.jpg")
    im.save(path)

In [ ]:
## Create batches of images from directory

def create_batches(images, batch_size=BATCH_SIZE):
    while True:
        for i in range(0, MAX_SIZE, batch_size):
            batch_images = image_files[i:i+batch_size]
            batch = np.array([imread(file_name) for file_name in batch_images])
            batch = (batch.astype(np.float32) - 127.5) / 127.5
            yield batch

In [ ]:
def create_checkpoint(generator, discriminator,epoch):
    path = os.path.join(os.environ['CHECKPOINT_PATH'], f"generator_epoch_{epoch}.h5")
    generator.save(path)

    path = os.path.join(os.environ['CHECKPOINT_PATH'], f"discriminator_epoch_{epoch}.h5")
    discriminator.save(path)


## Training the model

In [ ]:
real = np.ones((BATCH_SIZE, 1))
fake = np.zeros((BATCH_SIZE, 1))

discriminator_loss = []
generator_loss = []

image_files = np.array(image_files)

In [ ]:
reshapeModel = tf.keras.Sequential([layers.Resizing(128,128, input_shape=(218,178,3,))])

In [ ]:
for epoch in range(EPOCHS):

    print(f"----------- Epoch {epoch+1} -----------")

    epoch_images = np.random.choice(image_files, int(MAX_SIZE*RANDOM_SELECT), replace=False)

    imgGenerator = create_batches(epoch_images)

    discriminator_batch_loss = []
    discriminator_batch_acc = []
    generator_batch_loss = []

    for i in range (len(epoch_images)//BATCH_SIZE):

        imgs = next(imgGenerator)
        imgs = reshapeModel(imgs)

        noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))

        discriminator.trainable = True if (len(discriminator_loss) == 0) else discriminator_train(discriminator_loss[-1])

        gen_imgs = generator.predict(noise, verbose=0)

        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        discriminator.trainable = False

        g_loss = combined.train_on_batch(noise, real)

        discriminator_batch_loss.append(d_loss[0])
        generator_batch_loss.append(g_loss)
        discriminator_batch_acc.append(d_loss[1])

    discriminator_loss.append(np.mean(discriminator_batch_loss))
    generator_loss.append(np.mean(generator_batch_loss))


    print(f"Discriminator loss: {d_loss[0]}   Accuracy: {d_loss[1]}")
    print(f"Generator loss: {g_loss}")
    print('---------------------------------------')

    if((epoch+1) % INTERVAL==0):
        sample_image(epoch+1)
        create_checkpoint(generator, discriminator, epoch+1)

In [ ]:
generator.save(os.path.join(os.environ['MODELPATH'], 'generator.h5'))
discriminator.save(os.path.join(os.environ['MODELPATH'], 'discriminator.h5'))

In [ ]:
discriminator.trainable

In [ ]:
generator = tf.keras.models.load_model(os.path.join(os.environ['CHECKPOINT_PATH'], 'generator_epoch_1320.h5'))
noise = np.random.normal(0, 1, (1, 250))
gen_imgs = generator.predict(noise)
gen_imgs = 0.5 * gen_imgs + 0.5
plt.imshow(gen_imgs[0])